# Automate Routine Task
## Text File Automation: Practical Usage with Excel to Text

<img src = "pic_data/Dextract.png" width = "500" align="center">

<img src = "pic_data/elephant_logo.jpg" width = "300">

https://www.facebook.com/elephant.data.th/

In [ ]:
import pyDOE

### Import

In [ ]:
import numpy as np # array management
import openpyxl # excel communication

import pyDOE #Design Of Experiment Library

import pathlib # to get suffix of the file

###  Section#1 Prepare data

#### Input
- Load data from Excel file "INPUT_LHS_MANAGER.xlsx"
- Collect variable and collect in Python

#### Output
- Data Loaded in python
- Range of varible and thier type
- Latin Hypercube variable
- Template file name

In [ ]:
print('LHSMNG: Latin Hypercube Manager: EXCEL to text file')
print('LHSMNG: Developped by Peerapong E. et.al., pls contact peeraponge@gmail.com' )

"""
------------------------------------------------------------------------------
Section#1 Prepare data
------------------------------------------------------------------------------
"""

print('LHSMNG: Step-1 Reading data from EXCEL')

excelfilename = 'INPUT_LHS_MANAGER.xlsx'
excelsheetname = 'Input'

excelbookin = openpyxl.load_workbook(filename=excelfilename, data_only = True)
excelsheet  = excelbookin[excelsheetname]

CRIT = 'maximin' # 'maximin', 'center', 'centermaximin', 'correlation'

SFILE     = []
VARID     = []
SFILEID   = []
KEYWORD   = []
DTYPE     = []
DLOW      = []
DHIGH     = []

NVAR      = excelsheet.max_row - 11
NSAM      = excelsheet.cell(row = 2, column = 2).value
ITLHS     = excelsheet.cell(row = 3, column = 2).value
NSFILE    = excelsheet.cell(row = 4, column = 2).value

# Load file name
for i in range(5, 5 + NSFILE): 
    SFILE.append(excelsheet.cell(row = i, column = 2).value) ;


# Load varialbe
for i in range(12, excelsheet.max_row + 1):
    VARID.append(excelsheet.cell(row = i, column = 1).value)
    SFILEID.append(excelsheet.cell(row = i, column = 3).value)
    KEYWORD.append(excelsheet.cell(row = i, column = 4).value)
    DTYPE.append(excelsheet.cell(row = i, column = 5).value)
    DLOW.append(excelsheet.cell(row = i, column = 6).value)
    DHIGH.append(excelsheet.cell(row = i, column = 7).value)



#Convert to np array
VARID_np     = np.array(VARID) 
SFILEID_np   = np.array(SFILEID)
KEYWORD_np   = np.array(KEYWORD)
DTYPE_np     = np.array(DTYPE)
DLOW_np      = np.array(DLOW)
DHIGH_np     = np.array(DHIGH)


print('LHSMNG: Step-1 Completed')

#### Let's explore the result for step-1

In [ ]:
# List to containt file name
SFILE

In [ ]:
# Number of sample
NSAM 

In [ ]:
# Number of variable
NVAR 

In [ ]:
# Number of iteration in latin-hypercube sampling
ITLHS

In [ ]:
VARID_np

In [ ]:
SFILEID_np

In [ ]:
KEYWORD_np

In [ ]:
DTYPE_np

In [ ]:
# make type as float for further cal
DLOW_np = DLOW_np.astype(float)
DLOW_np

In [ ]:
DHIGH_np = DHIGH_np.astype(float)
DHIGH_np

## Section#2 Calculate LHS samples

#### Input are

- Data type - int, float (DTYPE_np)
- Upper bound (DHIGH_np)
- Lower bound (DLOW_np)
- Number of sample (NSAM)
- Number of variable (NVAR)

#### Output are
- Sample space array with size (NSAM * NVAR)

In [ ]:

"""
------------------------------------------------------------------------------
Section#2 Calculate LHS samples
------------------------------------------------------------------------------
"""
print('LHSMNG: Step-2 LHS Sampling and adjusting range')

X_LHS = pyDOE.lhs(NVAR, samples=NSAM,criterion=CRIT,iterations=ITLHS) # Generate LHS sample (0,1)





# modify int format data
DHIGH_np[DTYPE_np == 'int'] = DHIGH_np[DTYPE_np == 'int'] + 0.5
DLOW_np[DTYPE_np == 'int']  = DLOW_np[DTYPE_np == 'int']  - 0.5

# calculate range
DRANGE_np = DHIGH_np - DLOW_np


# prepare x data
DRANGE_TEMP_np  = np.repeat([DRANGE_np],NSAM,axis = 0)
DLOW_TEMP_np    = np.repeat([DLOW_np],NSAM,axis = 0)

X = DLOW_TEMP_np + (X_LHS * DRANGE_TEMP_np) # Calculate actual value

X[:,(DTYPE_np == 'int')] = np.round(X[:,(DTYPE_np == 'int')]).astype(int) # Modify for int variable

print('LHSMNG: Step-2 Complete')

#### Let's explore the result for step-2

In [ ]:
# Size is (Number of Sample, Number of attribute to change)
X.shape


In [ ]:
X

In [ ]:
# X แต่ละ ค่า คือ แต่ละ sample
X[0,:]

In [ ]:
# X แต่ละ ค่า คือ แต่ละ sample
X[1,:]

## Section#3 Assign "X" Modified LHS_Sample to text file

#### Input are

- X which is array containing experiment sample from Latin Hypercube
- template-1, template-2, ...., template-N : containing keyword to be modified

#### Output are
- Written template to replace variable inside

In [ ]:
"""
------------------------------------------------------------------------------
Section#3 Assign "X" Modified LHS_Sample to text file
------------------------------------------------------------------------------
"""
print('LHSMNG: Step-3 Modifying template file')


for i in range(0,NSFILE):
    #print(i)
    print('LHSMNG: Reading Source file : ', str(i+1),' from source : ', SFILE[i])
    FILE = open(SFILE[i], 'r') 
    READ = FILE.read()
    FILE.close()
    
    SUFFIX = pathlib.Path(SFILE[i]).suffix # suffix of the file
    

    
    KEYWORD_REP_np = KEYWORD_np[SFILEID_np == (i+1)]
    VARID_REP_np   = VARID_np[SFILEID_np == (i+1)] - 1

    for k in range(0,NSAM):
        WRITE = READ
        for j in range(0,len(VARID_REP_np)): # for each keyword

            WRITE = WRITE.replace(KEYWORD_REP_np[j],str(X[k,VARID_REP_np[j]])) # จริงๆ หลักๆ คือ line นี้ที่ทำการ replace ค่าครับ

        WNAME = 'OUTPUT_SOURCE-'+str(i+1)+ '_SAMPLE-' + str(k+1) + SUFFIX
        FILE = open(WNAME,'w')
        FILE.write(WRITE)
        FILE.close()


print('LHSMNG: Step-3 Complete')